# Modeling Approaches

In [1]:
# pip install xgboost

In [2]:
import pandas as pd
import numpy as np
# import dask.dataframe as dd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from collections import defaultdict
# Week 7
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import xgboost as xgb

## Reading into the data

In [3]:
pd.set_option('display.max_columns', None)

train = pd.read_csv('train_final.csv', low_memory=False)
validation = pd.read_csv('val_set_final.csv')

In [4]:
train.head()

,Unnamed: 0,date,customer_code,employee_index,country,female,age,new_cust,seniority_in_months,cust_type,residency_spain,birth_spain,join_channel,province_name,active_cust,income,segment,savings_acct,guarantees,current_acct,derivada_acct,payroll_acct,junior_acct,mas_particular_acct,particular_acct,particular_plus_acct,short_term_depo,medium_term_depo,long_term_depo,e_acct,funds,mortgage,pension,loans,taxes,credit_card,securities,home_acct,pensions_2,direct_debt,total_products,01 - TOP,02 - PARTICULARES,03 - UNIVERSITARIO,join_channel_encoded,province_name_encoded,employee_index_encoded,customer_code_encoded,income_to_age
0,0,2015-07-28,664160,N,1,0,0.632653,0,0.402344,1,1,0,KAR,MADRID,0,1.989686,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1.424185,1.749698,1.407278,263662,3.144938
1,1,2016-01-28,1076784,N,1,0,0.214286,0,0.152344,1,1,0,KHE,LERIDA,0,-0.306603,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0.886876,1.006139,1.407278,459750,-1.430748
2,2,2015-12-28,672465,N,1,0,0.387755,0,0.417969,1,1,0,KFC,SEVILLA,1,-0.148205,02 - PARTICULARES,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,4,0,1,0,1.559984,1.382030,1.407278,266890,-0.382203
3,3,2015-10-28,774528,N,1,0,0.397959,0,0.343750,1,1,0,KFA,MURCIA,1,-0.228531,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,1,0,1.850124,1.075147,1.407278,300528,-0.574243
4,4,2016-05-28,569598,N,1,0,0.459184,0,0.496094,1,1,0,KAT,MADRID,1,0.588748,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1.942077,1.749698,1.407278,227731,1.282133


In [5]:
validation.head()

,Unnamed: 0.1,Unnamed: 0,date,customer_code,employee_index,country_spain,female,age,first_contract_date,new_cust,seniority_in_months,primary_cust,last_date_primary,cust_type,residency_spain,birth_spain,join_channel,deceased,province_name,active_cust,income,segment,savings_acct,guarantees,current_acct,derivada_acct,payroll_acct,junior_acct,mas_particular_acct,particular_acct,particular_plus_acct,short_term_depo,medium_term_depo,long_term_depo,e_acct,funds,mortgage,pension,loans,taxes,credit_card,securities,home_acct,payroll_acct.1,pensions_2,direct_debt,total_products,01 - TOP,02 - PARTICULARES,03 - UNIVERSITARIO,join_channel_encoded,province_name_encoded,employee_index_encoded,income_to_age
0,0,0,2015-11-28,161428,N,1,1,0.744898,2000-03-16,0,0.738281,1,0,1,1,1,KAS,0,MADRID,0,-0.001810,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2.015705,1.749698,1.407278,-0.002430
1,1,1,2015-12-28,367478,N,1,1,0.418367,2002-07-02,0,0.632812,1,0,1,1,0,KAT,0,MADRID,1,1.473140,02 - PARTICULARES,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,5,0,1,0,1.942077,1.749698,1.407278,3.521079
2,2,2,2015-11-28,643150,N,1,0,0.520408,2006-09-21,0,0.429688,1,0,1,1,0,KFC,0,ALBACETE,0,-0.131613,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1.559984,1.103044,1.407278,-0.252899
3,3,3,2016-04-28,1385854,N,1,0,0.367347,2015-03-03,0,0.050781,1,0,1,1,0,KHK,0,SEVILLA,0,-0.056218,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1.122151,1.382030,1.407278,-0.153034
4,4,4,2015-08-28,495733,N,1,0,0.346939,2004-10-13,0,0.507812,1,0,1,1,0,KFC,0,MADRID,0,0.289349,02 - PARTICULARES,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,1.559984,1.749698,1.407278,0.833982


## Pre-processing  
Changing columns name and dropping columns so both datasets are the same

In [6]:
train = train.rename(columns={'country': 'country_spain'})

In [7]:
train = train.drop(columns=['Unnamed: 0'])
validation = validation.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
drop = ['join_channel', 'province_name', 'employee_index', 'segment', 'total_products']
train = train.drop(columns=drop + ['customer_code_encoded'])
validation = validation.drop(columns=drop + ['payroll_acct.1', 'first_contract_date', 'primary_cust', 'last_date_primary', 'deceased'])

Setting products we want to predict

In [8]:
products = ['savings_acct', 'guarantees', 'current_acct',
       'derivada_acct', 'payroll_acct', 'junior_acct', 'mas_particular_acct',
       'particular_acct', 'particular_plus_acct', 'short_term_depo',
       'medium_term_depo', 'long_term_depo', 'e_acct', 'funds', 'mortgage',
       'pension', 'loans', 'taxes', 'credit_card', 'securities', 'home_acct',
       'pensions_2', 'direct_debt']

Dropping duplicates on customer code column since the last instance will show all the products a client has 

In [9]:
train = train.drop_duplicates(subset=['customer_code'], keep='last')
validation = validation.drop_duplicates(subset=['customer_code'], keep='last')

# Removing customers from validation set that appear in training set
validation = validation[~validation['customer_code'].isin(train['customer_code'])]

In [10]:
print(train.info())
print(validation.info())

<class 'pandas.core.frame.DataFrame'>
Index: 706866 entries, 39288 to 6579716
Data columns (total 42 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   date                    706866 non-null  object 
 1   customer_code           706866 non-null  int64  
 2   country_spain           706866 non-null  int64  
 3   female                  706866 non-null  int64  
 4   age                     706866 non-null  float64
 5   new_cust                706866 non-null  int64  
 6   seniority_in_months     706866 non-null  float64
 7   cust_type               706866 non-null  int64  
 8   residency_spain         706866 non-null  int64  
 9   birth_spain             706866 non-null  int64  
 10  active_cust             706866 non-null  int64  
 11  income                  706866 non-null  float64
 12  savings_acct            706866 non-null  int64  
 13  guarantees              706866 non-null  int64  
 14  current_acct        

Defining our Xs and Ys

In [11]:
X_train = train.drop(['customer_code', 'date'] + products, axis=1)
y_train = train[products]

X_val = validation.drop(['customer_code', 'date'] + products, axis=1)
y_val = validation[products]

In [12]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

print("Shape of X_val:", X_val.shape)
print("Shape of y_val:", y_val.shape)

Shape of X_train: (706866, 17)
Shape of y_train: (706866, 23)
Shape of X_val: (200333, 17)
Shape of y_val: (200333, 23)


# Week 7 - Develop Second Modeling Approach

Setting hyperparameters for XGBoost model

In [13]:
# Define hyperparameter variations
hyperparameters = [
    {
        'max_depth': 3,
        'learning_rate': 0.1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
    },
    {
        'max_depth': 5,
        'learning_rate': 0.05,
        'subsample': 0.7,
        'colsample_bytree': 0.7,
    },
    {
        'max_depth': 7,
        'learning_rate': 0.01,
        'subsample': 0.6,
        'colsample_bytree': 0.6,
    }
]

Creating funtion to train and evaluate the XGBoost model  
  
This week we will keep the same metrics used last week -ROC AUC, F1 Score and Confusion Matrix, so we can compare in the end which model is better  
  
We are looping through ech one of the products, and creating a DMatrix to store the data. To avoid overfitting, we are using early stopping, which means that if the model does not improve for 20 rounds, it will stop early

In [14]:
# Model function
def xgb_model(X_train, y_train, X_val, y_val, params):
    metrics = {}
    for product in y_train.columns:
        # Creating DMatrix for XGBoost
        dtrain = xgb.DMatrix(X_train, label=y_train[product])
        dval = xgb.DMatrix(X_val, label=y_val[product])
        
        # Setting up early stopping to prevent overfitting
        early_stopping_rounds = 20
        eval_set = [(dtrain, 'train'), (dval, 'val')]
        
        # Train
        model = xgb.train(
            params,
            dtrain,
            num_boost_round=1000,
            evals=[(dval, 'val')],
            early_stopping_rounds=early_stopping_rounds,
            verbose_eval=False
        )
        
        # Predictions
        y_train_pred = model.predict(dtrain)
        y_val_pred = model.predict(dval)
        
        # Metrics
        metrics[product] = {
            'train': calc_metrics(y_train[product], y_train_pred),
            'val': calc_metrics(y_val[product], y_val_pred)
        }
    
    return metrics

# Metrics Function
def calc_metrics(y_true, y_pred):
    return {
        'ROC AUC': roc_auc_score(y_true, y_pred),
        'F1 Score': f1_score(y_true, y_pred > 0.5),
        'Confusion Matrix': confusion_matrix(y_true, y_pred > 0.5)
    }

In [15]:
# Calling the funtions to run the model
results = []
for i, params in enumerate(hyperparameters):
    metrics = xgb_model(X_train, y_train, X_val, y_val, params)
    
    avg_train_roc_auc = np.mean([metrics[product]['train']['ROC AUC'] for product in y_train.columns])
    avg_train_f1 = np.mean([metrics[product]['train']['F1 Score'] for product in y_train.columns])
    avg_val_roc_auc = np.mean([metrics[product]['val']['ROC AUC'] for product in y_train.columns])
    avg_val_f1 = np.mean([metrics[product]['val']['F1 Score'] for product in y_train.columns])
    
    results.append({
        'Variation': f'Variation {i+1}',
        'Train ROC AUC': avg_train_roc_auc,
        'Train F1 Score': avg_train_f1,
        'Val ROC AUC': avg_val_roc_auc,
        'Val F1 Score': avg_val_f1
    })


Creating a table to summarize results and define which variation is the best

In [28]:
# Summary table
results = pd.DataFrame(results)
print(results.to_string(index=False))

best_model = results.loc[results['Val ROC AUC'].idxmax()]['Variation']
print(f"\nThe best model for this week is {best_model}")

  Variation  Train ROC AUC  Train F1 Score  Val ROC AUC  Val F1 Score
Variation 1       0.918933        0.144641     0.904372      0.132815
Variation 2       0.926318        0.150042     0.923959      0.139032
Variation 3       0.933340        0.135798     0.884652      0.117693

The best model for this week is Variation 2


Creating a table to see the results in a easier to interpret way

In [19]:
train_metrics_list = []
val_metrics_list = []

for variation_idx, params in enumerate(hyperparameters, 1):
    variation_name = f"Variation {variation_idx}"
    
    for product in y_train.columns:

        train_roc_auc = metrics[product]['train']['ROC AUC']
        train_f1_score = metrics[product]['train']['F1 Score']
        train_conf_matrix = metrics[product]['train']['Confusion Matrix']

        val_roc_auc = metrics[product]['val']['ROC AUC']
        val_f1_score = metrics[product]['val']['F1 Score']
        val_conf_matrix = metrics[product]['val']['Confusion Matrix']
        
        # Appending lists
        train_metrics_list.append({
            'Variation': variation_name,
            'Dataset': 'Train',
            'Product': product,
            'ROC AUC': train_roc_auc,
            'F1 Score': train_f1_score,
            'Confusion Matrix': train_conf_matrix
        })
        

        val_metrics_list.append({
            'Variation': variation_name,
            'Dataset': 'Validation',
            'Product': product,
            'ROC AUC': val_roc_auc,
            'F1 Score': val_f1_score,
            'Confusion Matrix': val_conf_matrix
        })


metrics_df = pd.DataFrame(train_metrics_list + val_metrics_list)
print(metrics_df.to_string(index=False))

  Variation    Dataset              Product  ROC AUC  F1 Score                    Confusion Matrix
Variation 1      Train         savings_acct 0.993083  0.000000              [[706792, 0], [74, 0]]
Variation 1      Train           guarantees 0.999966  0.000000              [[706850, 0], [16, 0]]
Variation 1      Train         current_acct 0.808806  0.804876 [[171753, 105207], [69525, 360381]]
Variation 1      Train        derivada_acct 0.967359  0.000000             [[706603, 0], [263, 0]]
Variation 1      Train         payroll_acct 0.896745  0.013931        [[667803, 58], [38731, 274]]
Variation 1      Train          junior_acct 0.999975  0.966054         [[699891, 394], [64, 6517]]
Variation 1      Train  mas_particular_acct 0.981853  0.015199           [[701120, 1], [5701, 44]]
Variation 1      Train      particular_acct 0.925143  0.438586   [[602473, 17660], [57410, 29323]]
Variation 1      Train particular_plus_acct 0.921532  0.001422          [[677355, 3], [29487, 21]]
Variation 

## Generate product recommendations  
Here we want to visualize the product recommendations for each customer  
We will generate the product recommendations using Variation 2 - that had the best performance between all variations

In [31]:
best_params = hyperparameters[int(best_model.split()[-1]) - 1]

# Train model using Variation 2
product_models = {}
for product in y_train.columns:
    dtrain = xgb.DMatrix(X_train, label=y_train[product])
    model = xgb.train(
        best_params,
        dtrain,
        num_boost_round=1000,
        verbose_eval=False
    )
    product_models[product] = model

# Generate predictions
train_preds = {}
for product in y_train.columns:
    dtrain = xgb.DMatrix(X_train)
    proba = product_models[product].predict(dtrain)
    
    for customer_id, prob in zip(train['customer_code'], proba):
        if customer_id not in train_preds:
            train_preds[customer_id] = []
        train_preds[customer_id].append((product, prob))

We want to make sure that we are not recommending a product that the customer already own, so we will store the products that customers already have

In [32]:
def get_active_products(customer_data):
    return set(product for product in y_train.columns if customer_data[product] > 0)

We will sort the recommended products by the probability of a client getting it and we will get the top 7 recommendations

In [33]:
# Generate top 7 products, excluding owned products
for customer_id in train_preds:
    sorted_prods = sorted(train_preds[customer_id], key=lambda x: x[1], reverse=True)
    customer_data = train[train['customer_code'] == customer_id].iloc[0]
    active_products = get_active_products(customer_data)
    recommended_products = [prod for prod, _ in sorted_prods if prod not in active_products]
    
    # Top 7
    train_preds[customer_id] = recommended_products[:7]

# Recommendations for the first 5 customers
print("Recommendations:")
for customer_id in list(train_preds.keys())[:5]:
    print(f"Customer {customer_id}: {train_preds[customer_id]}")

Recommendations:
Customer 1225385: ['current_acct', 'taxes', 'pension', 'particular_acct', 'funds', 'credit_card', 'securities']
Customer 1358829: ['direct_debt', 'pensions_2', 'e_acct', 'payroll_acct', 'taxes', 'securities', 'pension']
Customer 1436539: ['pensions_2', 'payroll_acct', 'direct_debt', 'mas_particular_acct', 'e_acct', 'particular_acct', 'credit_card']
Customer 1448049: ['current_acct', 'payroll_acct', 'particular_plus_acct', 'mas_particular_acct', 'mortgage', 'pension', 'loans']
Customer 1396837: ['direct_debt', 'pensions_2', 'payroll_acct', 'e_acct', 'mas_particular_acct', 'taxes', 'long_term_depo']


Lastly, we want to identify which product has been recommended the most and least in the model

In [34]:
product_rec_counts = {product: sum(1 for recs in train_preds.values() if product in recs) for product in y_train.columns}
most_rec = max(product_rec_counts, key=product_rec_counts.get)
least_rec = min(product_rec_counts, key=product_rec_counts.get)

print(f"Most frequently recommended product: {most_rec} ({product_rec_counts[most_rec]} times)")
print(f"Least frequently recommended product: {least_rec} ({product_rec_counts[least_rec]} times)")

Most frequently recommended product: taxes (533949 times)
Least frequently recommended product: guarantees (18541 times)
